In [122]:
#解析生成json的内容
#解析决策过程的日志
import os
import json
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def paese_result_json(log_path):
    #读取json文件
    res=json.load(open(log_path,"r"))['update']
    #去除掉init_state部分
    static={}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        
        for action in actions:
            if str(action['class']) + '_' + str(action['id']) not in static:
                static[str(action['class']) + '_' + str(action['id'])]={}
            static[str(action['class']) + '_' + str(action['id'])][action['action_type']]=static[str(action['class']) + '_' + str(action['id'])][action['action_type']]+1 if action['action_type'] in static[str(action['class']) + '_' + str(action['id'])] else 1
            if 'damage' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['damage']=[]
            #print(action['class'] + '_' + str(action['id']),static[str(action['class']) + '_' + str(action['id'])]['damage'])

            if 'damage' in action:
                for damage in action['damage']:
                    static[str(action['class']) + '_' + str(action['id'])]['damage'].append(damage[1][0])                         
            
            
            
            if 'effects' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['effects']=[]

                        
#             if action['class'] + '_' + str(action['id'])=='monster_716666':
#                 print('77777', action['class'] + '_' + str(action['id']))
            
                
            if 'effects' in action:
                if len(action['effects'])>0:
                    for r in action['effects']:
                        static[str(action['class']) + '_' + str(action['id'])]['effects'].append(r[1])
        for state in states:
            if len(state)>0:
                for change in state:
                    if change[1][2]=="Hp":
                        #pass
                        if str(change[1][0]) + '_' + str(change[1][1]) not in static:
                            static[str(change[1][0]) + '_' + str(change[1][1])]={}
                        
                        if '最终HP' not in static[str(change[1][0]) + '_' + str(change[1][1])]:
                            static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=''
                        static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=change[2][1]

    for k in static.keys():
        if 'damage' not in static[k]:
            static[k]['damage']=[]
        if 'effects' not in static[k]:
            static[k]['effects']=[]
        static[k]['总伤害']=sum(static[k]['damage'])
        static[k]['总伤害次数']=len(static[k]['damage'])
        static[k]['平均伤害']=sum(static[k]['damage'])/len(static[k]['damage']) if len(static[k]['damage'])>0 else 0
        static[k]['各类效果']= list(set(num for sublist in static[k]['effects'] for num in sublist))
        if 'RIGHT' not in static[k]:
            static[k]['RIGHT']=0
        if 'LEFT' not in static[k]:
            static[k]['LEFT']=0
        if 'TOP' not in static[k]:
            static[k]['TOP']=0
        if 'BOTTOM' not in static[k]:
            static[k]['BOTTOM']=0
        static[k]['总移动次数']=static[k]['RIGHT']+static[k]['LEFT']+static[k]['TOP']+static[k]['BOTTOM']
        
        
        #隐藏具体伤害
        static[k].pop('damage')
        static[k].pop('effects')

        #按照key 排序
        static[k]=dict(sorted(static[k].items(),key=lambda x:x[0]))
    print('总tick',res[-1]['tick'])
    return static


In [123]:
log_files_path = os.getcwd()+"/test_data"
    #读取这个路径下所有文件
log_files = os.listdir(log_files_path)
for log_file in log_files:
    if log_file.endswith(".json"):
        log_path = log_files_path+"/"+log_file
        static_res_json=paese_result_json(log_path)
        df=pd.DataFrame(static_res_json).T
        display(df)

总tick 308


,BOTTOM,LEFT,MOVE_START,RIGHT,SKILL_77,TOP,WAIT,各类效果,平均伤害,总伤害,总伤害次数,总移动次数,最终HP,EFFECT_87,SKILL_88,SKILL_89,SKILL_90,SKILL_137,EFFECT_64,SKILL_100,SKILL_98,SKILL_99,SKILL_79,SKILL_80
monster_16714666,4,1,13,2,2,1,10,[],29.5,59,2,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_616666,3,0,3,5,2,0,NaN,[78],0.0,0,2,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_1316666,4,3,8,2,5,1,2,"[65, 67, 78]",20.2,101,5,10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_7115666,0,0,13,0,NaN,0,13,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hero_5003,1,5,11,16,NaN,11,NaN,[],247.272727,2720,11,33,277,3,3,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_716666,5,3,7,12,NaN,1,NaN,"[65, 67]",20.333333,122,6,21,0,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN
hero_5001,0,1,9,16,NaN,10,NaN,[],204.0,612,3,27,290,9,NaN,NaN,NaN,NaN,9,3,4,2,NaN,NaN
hero_5002,2,3,8,16,NaN,3,NaN,[],220.0,1320,6,24,408,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,2,6
monster_13713666,2,8,8,1,NaN,1,4,[],0,0,0,12,123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_13712666,3,0,6,3,NaN,0,4,[],4.0,4,1,6,0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
def parse_result_json_detail(log_path):
    res = json.load(open(log_path, "r"))['update']
    # 去除掉init_state部分
    static = {}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        for i in range(len(actions)):
            if actions[i]['id']==16714666:
                if actions[i]['action_type']!='MOVE_START' and actions[i]['action_type']!='WAIT':
                    print(actions[i],states[i],r['tick'])
log_files_path = os.getcwd() + "/test_data"
    # 读取这个路径下所有文件
log_path = log_files_path + "/for_qiangye_skill_"+str(99)+".json"
log_path = log_files_path + "/for_qiangye.json"

parse_result_json_detail(log_path)

{'action_type': 'BOTTOM', 'move_position': [16, 7, 13], 'id': 16714666, 'class': 'monster'} [['change', ['map', 0, 215, 'Block'], [1, 2]], ['change', ['map', 0, 216, 'Block'], [2, 1]], ['change', ['monster', 16714666, 'position', 2], [14, 13]]] 264
{'action_type': 'BOTTOM', 'move_position': [16, 7, 12], 'id': 16714666, 'class': 'monster'} [['change', ['map', 0, 214, 'Block'], [1, 2]], ['change', ['map', 0, 215, 'Block'], [2, 1]], ['change', ['monster', 16714666, 'position', 2], [13, 12]]] 264
{'action_type': 'RIGHT', 'move_position': [17, 7, 12], 'id': 16714666, 'class': 'monster'} [['change', ['map', 0, 214, 'Block'], [2, 1]], ['change', ['map', 0, 229, 'Block'], [1, 2]], ['change', ['monster', 16714666, 'position', 0], [16, 17]]] 264
{'action_type': 'LEFT', 'move_position': [16, 1, 11], 'id': 16714666, 'class': 'monster'} [['change', ['map', 0, 208, 'Block'], [1, 2]], ['change', ['map', 0, 228, 'Block'], [2, 1]], ['change', ['monster', 16714666, 'position', 0], [17, 16]], ['change', 

In [119]:
def parse_decision_log(log_path):
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'hero' in v:
                if v['hero']==5001:
                    if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                        if  v['stepname']=='决策树-首次查找最终选择' or  v['stepname']=='决策树-多次查找最终选择':
                            #if '判断是否满足恢复判定' in v['selection'] and v['result_len']=='True':
                                print(k,v)
                        
def static_decision_log(log_path):
    static={}
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                if  v['stepname']=='决策树-首次查找最终选择' and v['result_len']!=0:
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']][v['action']]=1
                    static[v['hero']][v['action']]=static[v['hero']][v['action']]+1
                    
                if  v['stepname']=='决策树-多次查找最终选择':
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']][v['action']]=1
                    static[v['hero']][v['action']]=static[v['hero']][v['action']]+1
                if v['stepname']=='决策树-偏好决策':
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']]['偏好_'+str(v['action'])]=1
                    static[v['hero']]['偏好_'+str(v['action'])]=static[v['hero']]['偏好_'+str(v['action'])]+1

    df=pd.DataFrame(static).T
    display(df)

In [120]:
#统计情况
static_decision_log(os.getcwd()+"/../log/logs.json")


,偏好_等待,等待,移动到敌人附近,移动到敌人附近并攻击,偏好_p_0_action,逃跑,移动到boss,寻找治疗目标并治疗
16714666,18.0,12.0,2.0,6.0,NaN,NaN,NaN,NaN
616666,2.0,NaN,2.0,5.0,NaN,NaN,NaN,NaN
1316666,6.0,3.0,2.0,3.0,NaN,NaN,NaN,NaN
7115666,18.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN
5003,2.0,NaN,NaN,2.0,2.0,5.0,2.0,NaN
5001,2.0,NaN,NaN,6.0,NaN,NaN,NaN,8.0
5002,2.0,NaN,NaN,5.0,NaN,NaN,3.0,5.0
13713666,11.0,9.0,2.0,NaN,NaN,2.0,NaN,NaN
13712666,10.0,8.0,2.0,2.0,NaN,NaN,NaN,NaN
8115666,11.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:

tmp_log_file = os.getcwd()+"/../log/logs.json"
parse_decision_log(tmp_log_file)

2024-10-08 16:31:25.125823_84 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '移动到敌人附近并攻击', 'hero': 5001}
2024-10-08 16:31:30.848956_326 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '寻找治疗目标并治疗', 'hero': 5001}
2024-10-08 16:31:43.152984_644 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '移动到敌人附近并攻击', 'hero': 5001}
2024-10-08 16:31:59.528933_896 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '移动到敌人附近并攻击', 'hero': 5001}
2024-10-08 16:32:22.004835_1107 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '移动到敌人附近并攻击', 'hero': 5001}
2024-10-08 16:33:05.382603_1268 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '寻找治疗目标并治疗', 'hero': 5001}
2024-10-08 16:34:24.674245_1442 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '寻找治疗目标并治疗', 'hero': 5001}
2024-10-08 16:35:44.439912_1591 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '寻找治疗目标并治疗', 'hero': 5001}
2024-10-08 16:37:44.501288_1769 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'ac